Failed Examples (53 total):
1, 3, 13, 14, 17, 21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 37, 41, 42, 43, 46, 48, 50, 51, 52, 53, 54, 56, 
62, 63, 65, 67, 68, 70, 72, 74, 77, 80, 81, 82, 83, 84, 85, 86, 87, 88,
89, 92, 93, 94, 95, 96, 99

In [2]:
xml_content = """<schema_linking>
  <query_decomposition>
    <output_terms>
      <term>how many schools</term>
    </output_terms>
    <constraint_terms>
      <term>Contra Costa</term>
      <term>directly funded</term>
      <term>number of test takers not more than 250</term>
    </constraint_terms>
    <evidence_constraints>
      <mathematical_formulas>COUNT(schools) where county is Contra Costa, funding type is directly funded, and number of test takers ≤ 250</mathematical_formulas>
      <calculation_requirements>Need to join school directory with SAT scores to filter by number of test takers; need funding type and county from school directory</calculation_requirements>
      <table_implications>Requires both schools and satscores tables; join on CDSCode/cds</table_implications>
    </evidence_constraints>
  </query_decomposition>

  <output_entity_mapping>
    <entity term="how many schools" confidence="high">
      <table name="schools">
        <column name="CDSCode" data_type="text">
          <typical_values>06344100630554, 06344100630562, ...</typical_values>
          <exact_match_value/>
          <match_reason>CDSCode is the unique identifier for each school; counting these gives the number of schools.</match_reason>
        </column>
      </table>
    </entity>
  </output_entity_mapping>

  <reduce_output_entities>
    <reduction_test>
      <original_columns>schools.CDSCode</original_columns>
      <single_column_test table="schools" column="CDSCode">
        <evaluation>yes</evaluation>
        <reasoning>Counting CDSCode uniquely identifies the number of schools matching the constraints.</reasoning>
      </single_column_test>
      <final_reduced_columns table="schools">CDSCode</final_reduced_columns>
      <overlap_analysis>No overlap; only CDSCode is needed for counting schools.</overlap_analysis>
    </reduction_test>
  </reduce_output_entities>

  <constraint_entity_mapping>
    <entity term="Contra Costa" confidence="high">
      <table name="schools">
        <column name="County" data_type="text">
          <typical_values>Los Angeles, San Diego, Orange, San Bernardino, Santa Clara, Alameda</typical_values>
          <exact_match_value>Contra Costa</exact_match_value>
          <match_reason>County column in schools table holds county names; filter for 'Contra Costa'.</match_reason>
        </column>
      </table>
    </entity>
    <entity term="directly funded" confidence="high">
      <table name="schools">
        <column name="FundingType" data_type="text">
          <typical_values>Directly funded, Locally funded, Not in CS funding model</typical_values>
          <exact_match_value>Directly funded</exact_match_value>
          <match_reason>FundingType column in schools table matches 'directly funded'.</match_reason>
        </column>
      </table>
    </entity>
    <entity term="number of test takers not more than 250" confidence="high">
      <table name="satscores">
        <column name="NumTstTakr" data_type="integer">
          <typical_values>Sample values not shown, but column name matches exactly.</typical_values>
          <exact_match_value/>
          <match_reason>NumTstTakr in satscores table is the number of test takers; filter for ≤ 250.</match_reason>
        </column>
      </table>
    </entity>
  </constraint_entity_mapping>

  <required_entity_discovery>
    <entity purpose="join" required_for="Linking school directory to SAT scores">
      <table name="schools">
        <column name="CDSCode" usage="join"/>
      </table>
      <table name="satscores">
        <column name="cds" usage="join"/>
      </table>
    </entity>
  </required_entity_discovery>

  <selected_tables>
    <table name="schools" alias="s" purpose="School directory and filtering by county and funding type">
      <columns>
        <column name="CDSCode" usage="select|join"/>
        <column name="County" usage="filter"/>
        <column name="FundingType" usage="filter"/>
      </columns>
    </table>
    <table name="satscores" alias="sat" purpose="SAT test taker filtering">
      <columns>
        <column name="cds" usage="join"/>
        <column name="NumTstTakr" usage="filter"/>
      </columns>
    </table>
  </selected_tables>
  
  <joins>
    <join from_table="schools" from_column="CDSCode" to_table="satscores" to_column="cds" type="INNER"/>
  </joins>
</schema_linking>"""

In [3]:
import xml.etree.ElementTree as ET

root = ET.fromstring(xml_content)